In [76]:
import ctf
from ctf import random
import numpy as np
import numpy.linalg as la
glob_comm = ctf.comm()

#### Symmetric Case 

In [25]:
# rank 3 tensor
n = 3
A = ctf.tensor((n,n,n),sp=True)
A.fill_sp_random(0,1,1)
A

array([[[ 0.85401372,  0.18131606,  0.26859406],
        [ 0.78908386,  0.65059516,  0.6627158 ],
        [ 0.        ,  0.26687633,  0.00357873]],

       [[ 0.24683768,  0.48001242,  0.70520176],
        [ 0.        ,  0.11745619,  0.46197907],
        [ 0.4593875 ,  0.        ,  0.        ]],

       [[ 0.09307546,  0.93264594,  0.28619528],
        [ 0.45564329,  0.78688555,  0.89040718],
        [ 0.        ,  0.63614484,  0.5163691 ]]])

In [57]:
[U,S,V] = ctf.svd(A.reshape((n*n,n)))
#print(ctf.vecnorm(A.reshape((n*n, n))-U@ctf.diag(S,sp=True)@V))
print(V.shape)

(3, 3)


In [71]:
ctf.random.seed(42)
Z1 = ctf.random.random((n,3))
Z2 = ctf.random.random((n,3))
Z3 = ctf.random.random((n,3))
lmbda = ctf.random.random((3))

niter = 0

def normalize(Z):
    norms = ctf.tensor(3)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

normalize(Z1)
normalize(Z2)
normalize(Z3)

E = ctf.tensor((n,n,n))
E.i("ijk") << A.i("ijk") - lmbda.i("u")*Z1.i("iu")*Z2.i("ju")*Z3.i("ku")
err_norm = ctf.vecnorm(E)

while (err_norm > 1.e-6 and niter < 100):
    if niter % 10 == 0:
        if glob_comm.rank() == 0:
            print(err_norm)
    M = ctf.tensor((n,n,3))
    M.i("jku") << Z2.i("ju")*Z3.i("ku")
    [U,S,V] = ctf.svd(M.reshape((n*n,3)))
    S = 1./S
    Z1.set_zero()
    Z1.i("iu") << V.i("vu")*S.i("v")*U.reshape((n,n,3)).i("jkv")*A.i("ijk")
    
    normalize(Z1)
    
    M.set_zero()
    M.i("iku") << Z1.i("iu")*Z3.i("ku")
    [U,S,V] = ctf.svd(M.reshape((n*n,3)))
    S = 1./S
    Z2.set_zero()
    Z2.i("ju") << V.i("vu")*S.i("v")*U.reshape((n,n,3)).i("ikv")*A.i("ijk")
    
    normalize(Z2)
    
    M.set_zero()
    M.i("iju") << Z1.i("iu")*Z2.i("ju")
    [U,S,V] = ctf.svd(M.reshape((n*n,3)))
    S = 1./S
    Z3.set_zero()
    Z3.i("ku") << V.i("vu")*S.i("v")*U.reshape((n,n,3)).i("ijv")*A.i("ijk")

    lmbda = normalize(Z3)
    
    E.set_zero()
    E.i("ijk") << A.i("ijk") - lmbda.i("u")*Z1.i("iu")*Z2.i("ju")*Z3.i("ku")
    err_norm = ctf.vecnorm(E)
    
    niter+=1

2.177316716788354
0.6273069180029907
0.5642149734890209
0.5371540234877374
0.5235373671243281
0.514717990408333
0.5080841718060254
0.5046922886451478
0.5032658405468982
0.5024500877083943


#### Non-symmetric case

In [75]:
# TODO: Tensor decomposition
def cp(T,I,J,K,r):
    '''
    Parameters
    ------
    T : Tensor to be approximated by CP decomposition
    r : estimated rank
    
    Returns
    -------
    factors : list of ndarray
        estimated low-rank decomposition (in kruskal tensor format)
    
    '''
    ctf.random.seed(42)
    U = ctf.random.random((I,r))
    V= ctf.random.random((J,r))
    W= ctf.random.random((K,r))
    lmbda = ctf.random.random((r))
    
    niter = 0
    
    def normalize(Z):
        norms = ctf.tensor(r)
        norms.i("u") << Z.i("pu")*Z.i("pu")
        norms = 1./norms**.5
        X = ctf.tensor(copy=Z)
        Z.set_zero()
        Z.i("pu") << X.i("pu")*norms.i("u")
        return 1./norms

    normalize(U)
    normalize(V)
    normalize(W)

    E = ctf.tensor((I,J,K))
    E.i("ijk") << T.i("ijk") - lmbda.i("u")*U.i("iu")*V.i("ju")*W.i("ku")
    err_norm = ctf.vecnorm(E)

    while (err_norm > 1.e-6 and niter < 100):
        if niter % 10 == 0:
            if glob_comm.rank() == 0:
                print(err_norm)
        
        M1 = ctf.tensor((J,K,r))
        M1.i("jku") << V.i("ju")*W.i("ku")
        [U_,S_,V_] = ctf.svd(M1.reshape((J*K,r)))
        S_ = 1./S_
        U.set_zero()
        U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
        normalize(U)
    
        M2 = ctf.tensor((I,K,r))
        M2.i("iku") << U.i("iu")*W.i("ku")
        [U_,S_,V_] = ctf.svd(M2.reshape((I*K,r)))
        S_ = 1./S_
        V.set_zero()
        V.i("ju") << V_.i("vu")*S_.i("v")*U_.reshape((I,K,r)).i("ikv")*T.i("ijk")
        normalize(V)
    
        M3 = ctf.tensor((I,J,r))
        M3.i("iju") << U.i("iu")*V.i("ju")
        [U_,S_,V_] = ctf.svd(M3.reshape((I*J,r)))
        S_ = 1./S_
        W.set_zero()
        W.i("ku") << V_.i("vu")*S_.i("v")*U_.reshape((I,J,r)).i("ijv")*T.i("ijk")
        lmbda = normalize(W)
    
        E.set_zero()
        E.i("ijk") << T.i("ijk") - lmbda.i("u")*U.i("iu")*V.i("ju")*W.i("ku")
        err_norm = ctf.vecnorm(E)
    
        niter+=1
        
    return U,V,W